* Lamida Function To push the file from one bucket to another as soon as it is inserted

In [ ]:
import json
import boto3

def lambda_handler(event, context):
    # Initialize S3 client
    s3 = boto3.client('s3')
    try:
        # Extract bucket name and object key from the event
        source_bucket = event['Records'][0]['s3']['bucket']['name']
        source_key = event['Records'][0]['s3']['object']['key']
        
        # Define destination bucket and key
        destination_bucket = 'zishtacoretransit-1'
        for _ in range(3):  # Retry up to 3 times
            try:
                # Copy the object to the destination bucket
                copy_source = {'Bucket': source_bucket, 'Key': source_key}
                s3.copy_object(CopySource=copy_source, Bucket=destination_bucket, Key=source_key)

                print(f"File copied from {source_bucket}/{source_key} to {destination_bucket}/{source_key}")
                
            except Exception as e:
                print(f"Error: {e}")
                raise e
        else:
                raise Exception("Max retries reached. File not found.")
    except Exception as e:
        print(f"Error: {e}")
        raise e
  
